<a href="https://colab.research.google.com/github/EmperorNao/MLHub/blob/main/DLS/sem2/HW1/%5Bhomework%5Dsimple_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2022-03-16 17:59:45--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  7.42MB/s    in 3m 0s   

2022-03-16 18:02:47 (7.71 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index2word)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [6]:
# method most_simmilar
wv_embeddings.most_similar('dog')

[('animal', 0.8564180135726929),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804807662964),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665),
 ('feline', 0.7392398118972778),
 ('bird', 0.7315489053726196),
 ('animal1', 0.7219215631484985),
 ('doggy', 0.7213349938392639),
 ('labrador', 0.7209131717681885)]

## Cat - нет, cats - да. Cats на 4 месте

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [5]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return map(str.lower, re.findall('\w+', text))
tokenizer = MyTokenizer()

In [6]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении
        
        return: векторное представление для вопроса
    """

    emb = np.array([embeddings[word] for word in tokenizer.tokenize(question) if word in embeddings])
    summary = np.zeros((1,dim)) if emb.shape[0] == 0 else emb

    return np.vstack([summary]).mean(axis = 0)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [21]:
print('%.2f' % question_to_vec("I love neural networks", wv_embeddings, tokenizer)[2])

-1.29


In [29]:
assert np.allclose(question_to_vec("one afgashfoa", wv_embeddings, tokenizer), question_to_vec("one", wv_embeddings, tokenizer))
assert np.allclose(question_to_vec("lashald fasldasdl", wv_embeddings, tokenizer), np.zeros(200))
assert not np.allclose(question_to_vec("lashald fasldasdl", wv_embeddings, tokenizer), question_to_vec("one", wv_embeddings, tokenizer))
assert not np.allclose(question_to_vec("not one", wv_embeddings, tokenizer), question_to_vec("one two", wv_embeddings, tokenizer))
assert np.allclose(question_to_vec("two one", wv_embeddings, tokenizer), question_to_vec("one two", wv_embeddings, tokenizer))

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

$min \space {DCG@1} = 0$

$max \space {Hits@47} = 1$

Ответ: 1

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [31]:
from math import log

print("%.1f" % (1.0 / log(10, 2)))

0.3


### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [7]:
import numpy as np

In [8]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    assert len(dup_ranks)

    return np.sum(np.array(dup_ranks) <= k) / len(dup_ranks)

In [9]:
def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    assert len(dup_ranks)

    r = np.array(dup_ranks)
    return np.sum(1 / np.log2(1 + r[r <= k])) / len(dup_ranks)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [10]:
import pandas as pd

In [11]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [candidates_ranking[0].index(copy_answers[0]) + 1]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [12]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [41]:
!wget -O stackoverflow_similar_questions.zip https://vk.com/doc209646771_632623356?hash=e43566f0f3772e2934&dl=cfcea5cc790953ebf1

--2022-03-16 18:06:41--  https://vk.com/doc209646771_632623356?hash=e43566f0f3772e2934
Resolving vk.com (vk.com)... 87.240.190.67, 87.240.190.72, 87.240.190.78, ...
Connecting to vk.com (vk.com)|87.240.190.67|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://psv4.userapi.com/c536132/u209646771/docs/d6/0514499e9e74/stackoverflow_similar_questions_zip.txt?extra=Hf1iGw24JEr-S4lNbKgKBJqVB1PZseAMCtBPiAh7sdhGRb6RMP1OhcIqgbx0yFwNr5Ka44XDF_OpD5yPNnt7mQPOwbgIZoUMXvQ8PO3fWpK7QpYqhM33tvsLoWKeVoc9vMz5jViSBBdugZ4&dl=1 [following]
--2022-03-16 18:06:43--  https://psv4.userapi.com/c536132/u209646771/docs/d6/0514499e9e74/stackoverflow_similar_questions_zip.txt?extra=Hf1iGw24JEr-S4lNbKgKBJqVB1PZseAMCtBPiAh7sdhGRb6RMP1OhcIqgbx0yFwNr5Ka44XDF_OpD5yPNnt7mQPOwbgIZoUMXvQ8PO3fWpK7QpYqhM33tvsLoWKeVoc9vMz5jViSBBdugZ4&dl=1
Resolving psv4.userapi.com (psv4.userapi.com)... 87.240.137.134, 87.240.137.140, 87.240.190.76, ...
Connecting to psv4.userapi.com (psv4.userapi.com)|87.24

In [42]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [13]:
def read_corpus(filename):
    data = []
    counter = 0
    for line in open(filename, encoding='utf-8'):
        data.append(line.split('\t'))
    return data

Нам понадобиться только файл validation.

In [14]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [47]:
len(validation_data)

3760

Размер нескольких первых строк

In [48]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [16]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """

    question_embedding = np.expand_dims(question_to_vec(question, embeddings, tokenizer), 0)
    candidates_embeddings = [np.expand_dims(question_to_vec(candidate, embeddings, tokenizer), 0) for candidate in candidates]

    pairs = [(i, cosine_similarity(question_embedding, embedding)) for i, embedding in enumerate(candidates_embeddings)]
    return list(reversed(sorted(pairs, key=lambda x: x[1])))

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [17]:
questions = ['converting string to list', 'Sending array via Ajax fails'] 

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],
              
              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [18]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, array([[0.5922089]], dtype=float32)), (0, array([[0.5414996]], dtype=float32)), (2, array([[0.09703708]], dtype=float32))]

[(0, array([[0.4613238]], dtype=float32)), (2, array([[0.32301265]], dtype=float32)), (1, array([[0.3005938]], dtype=float32))]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [53]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(1, 'Getting all list items of an unordered list in PHP'), #скрыт
            (0, 'select2 not displaying search results'), #скрыт
            (2, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

In [54]:
102

102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [25]:
from tqdm.notebook import tqdm

In [ ]:
wv_ranking = []
max_validation_examples = 10000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [ ]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

### Эмбеддинги, обученные на корпусе похожих вопросов

In [19]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [20]:
from gensim.models import Word2Vec

In [21]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import WordPunctTokenizer

nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [22]:
def pipeline(sent_preprocessor, window: int = 5):

    sentences = [sent_preprocessor(sent[0] + sent[1]) for sent in train_data]

    embeddings_trained = Word2Vec(sentences, # data for model to train on
                 size=200,                 # embedding vector size
                 min_count=5,             # consider words that occured at least 5 times
                 window=window).wv

    wv_ranking = []
    max_validation_examples = 10000
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)

    for k in tqdm([1, 5, 10, 100, 500, 1000]):
        print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

In [23]:
class Preprocessor:

    def __init__(self, tokenizer, lemmatizer = None):

        assert tokenizer
        self.tokenizer = tokenizer
        self.lemmatizer = lemmatizer


    def __call__(self, sentence):

        if self.lemmatizer:
            return [self.lemmatizer.lemmatize(word) for word in self.tokenizer.tokenize(sentence)]
        else:
            return [word for word in self.tokenizer.tokenize(sentence)]

In [26]:
simple_tokenizer = Preprocessor(WordPunctTokenizer())
pipeline(simple_tokenizer)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.245 | Hits@   1: 0.245
DCG@   5: 0.324 | Hits@   5: 0.395
DCG@  10: 0.345 | Hits@  10: 0.463
DCG@ 100: 0.396 | Hits@ 100: 0.714
DCG@ 500: 0.422 | Hits@ 500: 0.916
DCG@1000: 0.431 | Hits@1000: 1.000


In [27]:
tokenizer_and_lemmatizer = Preprocessor(WordPunctTokenizer(), lemmatizer = WordNetLemmatizer())
pipeline(tokenizer_and_lemmatizer)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.221 | Hits@   1: 0.221
DCG@   5: 0.298 | Hits@   5: 0.366
DCG@  10: 0.319 | Hits@  10: 0.431
DCG@ 100: 0.369 | Hits@ 100: 0.681
DCG@ 500: 0.397 | Hits@ 500: 0.898
DCG@1000: 0.408 | Hits@1000: 1.000


In [32]:
simple_tokenizer = Preprocessor(WordPunctTokenizer())
pipeline(simple_tokenizer, window=3)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.227 | Hits@   1: 0.227
DCG@   5: 0.304 | Hits@   5: 0.370
DCG@  10: 0.325 | Hits@  10: 0.434
DCG@ 100: 0.376 | Hits@ 100: 0.687
DCG@ 500: 0.404 | Hits@ 500: 0.904
DCG@1000: 0.414 | Hits@1000: 1.000


In [33]:
simple_tokenizer = Preprocessor(WordPunctTokenizer())
pipeline(simple_tokenizer, window=7)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.253 | Hits@   1: 0.253
DCG@   5: 0.334 | Hits@   5: 0.406
DCG@  10: 0.357 | Hits@  10: 0.476
DCG@ 100: 0.408 | Hits@ 100: 0.728
DCG@ 500: 0.432 | Hits@ 500: 0.921
DCG@1000: 0.441 | Hits@1000: 1.000


In [34]:
simple_tokenizer = Preprocessor(WordPunctTokenizer())
pipeline(simple_tokenizer, window=10)

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.264 | Hits@   1: 0.264
DCG@   5: 0.344 | Hits@   5: 0.416
DCG@  10: 0.366 | Hits@  10: 0.485
DCG@ 100: 0.417 | Hits@ 100: 0.737
DCG@ 500: 0.442 | Hits@ 500: 0.924
DCG@1000: 0.450 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Эмбеддинги, обученный на корпусе похожих вопросов лучше, потому что многие слова в данной теме специфичны, контекст слов специфичен.

Лучше всего работает на окне = 3, видимо так удаётся обучиться на достаточно большом количестве контекста без размазывания по большому количеству слов

Почему-то нормализация показывает метрики dcg в 2 раза хуже. Либо я что-то сделал не так, либо в данной задаче очень сильно влияют окончания

Плохое качество задачи - потому что мы не особо учитываем полный смысл предложений - усреднение векторов эмбеддингов не кажется самой лучшей идеей. Кажется, что можно было бы обучить сетку/линейную модель, которая бы принимала эмбеддинги и училась находить эмбеддинг предложения получше.